In [1]:
import pandas as pd

# Assuming the file "last_1k_rows.parquet" is in the current directory
file_path = "last_1k_rows.parquet"

# Read the Parquet file into a DataFrame
df = pd.read_parquet(file_path)

# Now you can work with the DataFrame 'df'


In [2]:
df

,company_id,company_name,short_description,long_description,batch,status,tags,location,country,year_founded,num_founders,founders_names,team_size,website,cb_url,linkedin_url,vector
0,28425,Pando Bioscience,Decipher complex diseases using protein networks,Pando Bioscience is a Boston-based synthetic b...,W23,Active,"['Drug discovery', 'Biotech', 'Diagnostics', '...",Boston,US,2022.0,2,"['Will (Yangxiaolu) Cao', 'Yang Wang']",3.0,https://,None,None,"[-0.03582763671875, 0.0177154541015625, 0.0542..."
1,28421,Apollo Group,A marketplace for consumers to hire trained bl...,Apollo Group is a diversified technology group...,W23,Active,"['Home Services', 'International', 'Marketplac...","Lahore, Pakistan",PK,2023.0,1,['Usman Gul'],8.0,http://www.apollo-group.io,https://www.crunchbase.com/organization/apollo...,https://www.linkedin.com/in/gulsf/,"[-0.0245361328125, 0.0084686279296875, 0.05404..."
2,28416,Pierre,A new way to review code,"Pierre enables engineers, designers and busine...",W23,Active,"['Developer Tools', 'Collaboration', 'AI-Enhan...",San Francisco,US,2023.0,2,"['Ian Ownbey', 'Jacob Thornton']",2.0,https://heypierre.app,None,None,"[-0.040374755859375, 0.015228271484375, 0.0489..."
3,28415,moonrepo,A developer productivity platform for managing...,moonrepo is a developer tool that reduces buil...,W23,Active,"['Developer Tools', 'SaaS', 'Open Source', 'En...","Portland, OR",US,2022.0,2,"['Miles Johnson', 'James Pozdena']",2.0,https://moonrepo.dev,None,https://www.linkedin.com/company/moonrepo/,"[-0.0469970703125, 0.0377197265625, 0.04132080..."
4,28414,Lasso,robotic process automation for chrome using GP...,🧐 The problem: Traditional Robotic Process Aut...,W23,Active,"['Generative AI', 'B2B', 'SaaS', 'Developer To...",San Francisco,US,2023.0,2,"['Gautam Bose', 'Lucas Ochoa']",2.0,https://www.getlassoai.com/,None,None,"[-0.0269012451171875, 0.07025146484375, 0.0514..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3790,379,Reddit,The frontpage of the internet.,Founded by Steve Huffman and Alexis Ohanian in...,S05,Acquired,"['Community', 'Social', 'Social Media', 'Socia...",San Francisco,US,NaN,1,['Steve Huffman'],201.0,http://reddit.com,https://www.crunchbase.com/organization/reddit,None,"[-0.032073974609375, 0.054962158203125, 0.0216..."
3791,378,Kiko,We're the best online calendar solution to eve...,Kiko is a web calendar for anyone who wants to...,S05,Acquired,['Calendar'],"Cambridge, MA",US,2005.0,1,['Emmett Shear'],0.0,http://kiko.com,https://www.crunchbase.com/organization/kiko,None,"[-0.0309295654296875, 0.0313720703125, 0.03060..."
3792,375,Clickfacts,None,ClickFacts is a Malware Software Solutions com...,S05,Acquired,[],San Francisco,US,NaN,2,"['Mikhail Ledvich', 'greg gurevich']",11.0,http://clickfacts.com,https://www.crunchbase.com/organization/clickf...,None,"[-0.0258941650390625, 0.0218353271484375, 0.04..."
3793,374,TextPayMe,None,"Founded in 2005, TextPayMe is an SMS payment s...",S05,Acquired,[],"Redmond, WA",US,NaN,1,['Phil Yuen'],11.0,http://textpayme.com,https://www.crunchbase.com/organization/textpayme,None,"[-0.03143310546875, 0.04669189453125, 0.052093..."


In [3]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import random
import requests
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm

api_key = "TWfVrlX659GSTS9hcsgUcPZ8uNzfoQsg"
model = "mistral-small-latest"
url = "https://api.mistral.ai/v1/embeddings"

client = MistralClient(api_key=api_key)

def generate_numbers(input_number, range_start, range_end, num_output=5):
    output_numbers = set()
    while len(output_numbers) < num_output:
        random_number = random.randint(range_start, range_end)
        if random_number != input_number:
            output_numbers.add(random_number)
    return list(output_numbers)

num_qad_pairs = len(df)
columns = {'question': [], 'answer': [], 'golden': [], 'distractor': []}
for n in tqdm(range(num_qad_pairs)):
    company_name = df.iloc[n]['company_name']
    messages = [
        ChatMessage(role="user", content="Summarize this into ten descriptor words and only print out the ten words, comma separated. " + 
    df.iloc[n]['long_description'])
    ]
    chat_response = client.chat(
        model=model,
        messages=messages,
    )
    chat_out = chat_response.choices[0].message.content
    if chat_out[-1] == '.':
        chat_out = chat_out[:-1]
    
    response = client.embeddings(
      model="mistral-embed",
      input=list(chat_out.split(", ")),
      )
    similarities = cosine_similarity([response.data[0].embedding], df['vector'].tolist())

    # Find the index of the closest embedding (highest cosine similarity)
    closest_index = np.argmax(similarities)

    # Get the embeddings corresponding to these indices
    closest_embedding = df.loc[closest_index, 'company_name']
    
    random_embeddings = []
    for i in generate_numbers(n, 0, 999, 5):
        random_embeddings.append(df.iloc[i]['long_description'])

    
    out_df = pd.DataFrame(columns=['question', 'answer', 'golden', 'distractor'])
    data = []
    if (closest_embedding == company_name):
        columns['question'].append('What company is best described using the following words? ' + chat_out)
        columns['answer'].append(company_name)
        columns['golden'].append(df.iloc[n]['long_description'])
        columns['distractor'].append(random_embeddings)
out_df = pd.DataFrame(columns)
out_df.to_csv('data.csv', index=False)

100%|██████████████████████████████████████████████| 3795/3795 [1:21:22<00:00,  1.29s/it]
